In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from tensorflow import keras

In [ ]:
data = pd.read_csv('FULLUwi.csv')

In [ ]:
data.head()

In [ ]:
list_features = data.columns
print('They are',len(list_features),'features in the dataset.')
print('----------------')
for f in list_features:
    print('feature:', f, '|| Type:', type(data[f][0]), '|| Example:', data[f][0], '|| number of unique values', len(data[f].unique()) )

They are 14 features in the dataset.
----------------
feature: Subject || Type: <class 'str'> || Example: ACCT || number of unique values 194
feature: CourseCode || Type: <class 'numpy.int64'> || Example: 1005 || number of unique values 1256
feature: CourseType || Type: <class 'str'> || Example: E || number of unique values 17
feature: CourseType1 || Type: <class 'str'> || Example: E11 || number of unique values 477
feature: Semester || Type: <class 'numpy.int64'> || Example: 1 || number of unique values 4
feature: Year || Type: <class 'str'> || Example: 2021/2022 || number of unique values 5
feature: Faculty || Type: <class 'str'> || Example: FSS || number of unique values 36
feature: Level || Type: <class 'str'> || Example: 1 || number of unique values 11
feature: Location || Type: <class 'str'> || Example: Mona - Weekend || number of unique values 18
feature: Lecturer || Type: <class 'str'> || Example: Paul, Dwayney (Primary)  || number of unique values 4946
feature: Students || Typ

In [ ]:
data.isnull().sum()

Subject            0
CourseCode         0
CourseType         2
CourseType1        0
Semester           0
Year               0
Faculty          718
Level              0
Location           0
Lecturer        4247
Students           0
Seats              0
Room             718
Attribute      50160
dtype: int64

In [ ]:
data['Room'].unique()

In [ ]:
def extraLec(name):
  if len(name.split())>3:
    return 'Yes'
  else:
    return 'No'

In [ ]:
values = {'CourseType':'None','Faculty':'UN','Lecturer':'UN','Room':'UN','Attribute':'None'}
data= data.fillna(value=values)
data['Lecturer2'] = data.apply(lambda x: extraLec(x.Lecturer), axis = 1)
data['Lecturer']  = data.apply(lambda x: ' '.join(x.Lecturer.split(' (Primary)')[0].split()[-2:]), axis=1) #6.8350

In [ ]:
column = ['Attribute']
dataframe = data.loc[:,~data.columns.isin(column)].copy()
dataframe['Room'] = pd.Categorical(data['Room']).codes
labels = dataframe.pop("Room")
feature_space = tf.keras.utils.FeatureSpace(
    features={
        "Subject": "string_categorical",
        "CourseCode": "integer_categorical",
        "CourseType": "string_categorical",
        "CourseType1": "string_categorical",
        "Semester": "integer_categorical",
        "Year": "string_categorical",
        "Level": "string_categorical",
        "Location": "string_categorical",
        "Lecturer": "string_categorical",
        "Lecturer2": "string_categorical",
        "Faculty": "string_categorical",
        "Students": "float_normalized",
        "Seats": "float_normalized"
    },
    output_mode="concat",
)
dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))
# Before you start using the FeatureSpace,
# you must `adapt()` it on some data.
feature_space.adapt(dataset)

# You can call the FeatureSpace on a dict of data (batched or unbatched).
output_vector = feature_space(dict(dataframe))

In [ ]:
X_train = output_vector[:40000]
X_test = output_vector[40000:]
y_train = labels[:40000]
y_test = labels[40000:]

In [ ]:
tf.random.set_seed(42)
model = keras.models.Sequential([
 keras.layers.Dense(500, kernel_initializer = 'lecun_normal', activation="selu",  input_shape=X_train.shape[1:]),
 keras.layers.Dense(229, activation="softmax")
])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
 restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)
opt = keras.optimizers.Nadam(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer = 'sgd', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=300,
 validation_data=(X_test, y_test), callbacks=[early_stopping_cb,model_checkpoint_cb])

Epoch 1/300
1250/1250 [==============================] - 27s 21ms/step - loss: 1.5799 - accuracy: 0.7390 - val_loss: 2.2551 - val_accuracy: 0.5764
Epoch 2/300
1250/1250 [==============================] - 27s 22ms/step - loss: 1.0870 - accuracy: 0.7944 - val_loss: 2.0570 - val_accuracy: 0.6034
Epoch 3/300
1250/1250 [==============================] - 26s 21ms/step - loss: 0.9842 - accuracy: 0.8045 - val_loss: 1.9415 - val_accuracy: 0.6236
Epoch 4/300
1250/1250 [==============================] - 25s 20ms/step - loss: 0.9150 - accuracy: 0.8115 - val_loss: 1.8536 - val_accuracy: 0.6373
Epoch 5/300
1250/1250 [==============================] - 25s 20ms/step - loss: 0.8588 - accuracy: 0.8204 - val_loss: 1.7805 - val_accuracy: 0.6487
Epoch 6/300
1250/1250 [==============================] - 25s 20ms/step - loss: 0.8102 - accuracy: 0.8263 - val_loss: 1.7156 - val_accuracy: 0.6570
Epoch 7/300
1250/1250 [==============================] - 25s 20ms/step - loss: 0.7666 - accuracy: 0.8335 - val_loss: 1

In [ ]:
np.argmax(model.predict(output_vector[:6]), axis = 1)

In [ ]:
k = 3  # Change k to the desired number of top predictions
top_indices = np.argsort(predictions[0])[::-1][:k]
for i in range(k):
    class_index = top_indices[i]
    class_probability = predictions[0][class_index]
    print(f"Prediction {i+1}: Class {pd.Categorical(data['Room']).categories[class_index]}, Probability: {class_probability:.2f}")

In [ ]:
prediction = np.argmax(model.predict(output_vector), axis = 1)

1884/1884 [==============================] - 26s 14ms/step


In [ ]:
data['PreRoom'] = prediction
data['PreRoom'] = data.apply(lambda x: pd.Categorical(data['Room']).categories[x.PreRoom],axis=1)
data.to_csv('PTest.csv',index=False)